## Setup

In [4]:
import pandas as pd
import numpy as np
import yfinance as yf

## Target Variable: Nasdaq Tech-100 Index

In [5]:
tech_ticker = "^NDXT"
nasdaq_t100_data = yf.download(tech_ticker, start="2020-03-01", end="2024-03-01")

[*********************100%%**********************]  1 of 1 completed


Let us examine the data

In [6]:
nasdaq_t100_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-03-02,5189.259766,5315.589844,5082.200195,5314.009766,5314.009766,551400
2020-03-03,5308.140137,5382.939941,5093.879883,5138.990234,5138.990234,599100
2020-03-04,5227.140137,5367.709961,5180.319824,5367.529785,5367.529785,414400
2020-03-05,5209.359863,5314.810059,5158.540039,5197.419922,5197.419922,408200
2020-03-06,5031.310059,5122.839844,4958.080078,5086.419922,5086.419922,523400
...,...,...,...,...,...,...
2024-02-23,10359.089844,10384.919922,10232.559570,10265.709961,10265.709961,0
2024-02-26,10313.870117,10389.429688,10282.669922,10329.919922,10329.919922,0
2024-02-27,10370.900391,10422.419922,10315.009766,10347.509766,10347.509766,0


We only want the closing price. So we will discard all other columns, and only keep the closing price column

In [7]:
nasdaq_t100_prices = nasdaq_t100_data[['Adj Close']]
nasdaq_t100_prices

,Adj Close
Date,
2020-03-02,5314.009766
2020-03-03,5138.990234
2020-03-04,5367.529785
2020-03-05,5197.419922
2020-03-06,5086.419922
...,...
2024-02-23,10265.709961
2024-02-26,10329.919922
2024-02-27,10347.509766


**To-Do**: Might need interpolation, to add prices for non-trading days.

## Feature Variable 1: Short term interest rates

We will use the secondary market rates of 3-Month Treasury Bills, calculated on a discount basis. These rates represent the interest rates at which these Treasury Bills are traded in the secondary market. Changes in the 3-Month Treasury Bill rates are considered to be indicators of shifts in the short-term interest rates.

Download source: https://fred.stlouisfed.org/series/DTB3 

In [8]:
short_term_rates_df = pd.read_csv("DTB3.csv")

Lets get a overview of how the data looks like.

In [9]:
short_term_rates_df

,DATE,DTB3
0,2020-03-02,1.10
1,2020-03-03,0.93
2,2020-03-04,0.71
3,2020-03-05,0.61
4,2020-03-06,0.45
...,...,...
1040,2024-02-26,5.26
1041,2024-02-27,5.25
1042,2024-02-28,5.24
1043,2024-02-29,5.25


Now we want the dates to be used as an index, instead of the sequence number, so we can match values across variables. So we transform the data to have date column as index instead.

In [10]:
# Maintaining consistent name across different dataframes
short_term_rates_df.rename(columns={"DATE":"Date"}, inplace=True)

# Change index to date
short_term_rates_df.set_index('Date', inplace=True)
short_term_rates_df.index = pd.to_datetime(short_term_rates_df.index)
short_term_rates_df

,DTB3
Date,
2020-03-02,1.10
2020-03-03,0.93
2020-03-04,0.71
2020-03-05,0.61
2020-03-06,0.45
...,...
2024-02-26,5.26
2024-02-27,5.25
2024-02-28,5.24


## Feature Variable 2: Long Term Interest Rates
We will use the Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis. <br>
It reflects the interest rate at which these Treasury securities are traded in the market. <br>
Changes in the these rates are considered to be indicators of shifts in the long-term interest rates.

Download source: https://fred.stlouisfed.org/series/DTB3 

In [11]:
long_term_rates_df = pd.read_csv("DGS10.csv")

Lets get a overview of how the data looks like.

In [12]:
long_term_rates_df

,DATE,DGS10
0,2020-03-02,1.10
1,2020-03-03,1.02
2,2020-03-04,1.02
3,2020-03-05,0.92
4,2020-03-06,0.74
...,...,...
1040,2024-02-26,4.28
1041,2024-02-27,4.31
1042,2024-02-28,4.27
1043,2024-02-29,4.25


Now we want the dates to be used as an index, instead of the sequence number, so we can match values across variables.

In [13]:
# Maintaining consistent name across different dataframes
long_term_rates_df.rename(columns={"DATE":"Date"}, inplace=True)

# Change index to date
long_term_rates_df.set_index('Date', inplace=True)
long_term_rates_df.index = pd.to_datetime(long_term_rates_df.index)
long_term_rates_df

,DGS10
Date,
2020-03-02,1.10
2020-03-03,1.02
2020-03-04,1.02
2020-03-05,0.92
2020-03-06,0.74
...,...
2024-02-26,4.28
2024-02-27,4.31
2024-02-28,4.27


## Feature Variable 3: US Dollar Currency strength

We will use the ICE US Dollar Index C as a proxy for the strength of US Dollars.

In [14]:
dollar_index_ticker = "DX-Y.NYB"
dollar_index_data = yf.download(dollar_index_ticker, start="2020-03-01", end="2024-03-01")

[*********************100%%**********************]  1 of 1 completed


Lets have a look at the data.

In [15]:
dollar_index_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-03-02,98.080002,98.089996,97.180000,97.360001,97.360001,0
2020-03-03,97.559998,97.699997,96.980003,97.150002,97.150002,0
2020-03-04,97.169998,97.589996,97.120003,97.339996,97.339996,0
2020-03-05,97.379997,97.440002,96.519997,96.820000,96.820000,0
2020-03-06,96.610001,96.699997,95.709999,95.949997,95.949997,0
...,...,...,...,...,...,...
2024-02-23,103.949997,104.050003,103.769997,103.940002,103.940002,0
2024-02-26,103.959999,104.019997,103.709999,103.830002,103.830002,0
2024-02-27,103.779999,103.919998,103.610001,103.830002,103.830002,0


We only want the closing price. So we will discard all other columns, and only keep the closing price column

In [16]:
dollar_index_values = dollar_index_data[['Adj Close']]
dollar_index_values

,Adj Close
Date,
2020-03-02,97.360001
2020-03-03,97.150002
2020-03-04,97.339996
2020-03-05,96.820000
2020-03-06,95.949997
...,...
2024-02-23,103.940002
2024-02-26,103.830002
2024-02-27,103.830002


## Feature Variable 4: CBOE Volatility Index

In [17]:
vix_ticker = "^VIX"
vix_data = yf.download(vix_ticker, start="2020-03-01", end="2024-03-01")

[*********************100%%**********************]  1 of 1 completed


Lets have a look at the data.

In [18]:
vix_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-03-02,38.540001,40.410000,31.500000,33.419998,33.419998,0
2020-03-03,33.650002,41.060001,24.930000,36.820000,36.820000,0
2020-03-04,34.439999,35.580002,30.299999,31.990000,31.990000,0
2020-03-05,33.610001,42.840000,33.540001,39.619999,39.619999,0
2020-03-06,48.930000,54.389999,40.840000,41.939999,41.939999,0
...,...,...,...,...,...,...
2024-02-23,14.310000,14.310000,13.640000,13.750000,13.750000,0
2024-02-26,14.170000,14.200000,13.660000,13.740000,13.740000,0
2024-02-27,13.630000,13.750000,13.410000,13.430000,13.430000,0


In [19]:
vix_index_values = vix_data[['Adj Close']]
vix_index_values

,Adj Close
Date,
2020-03-02,33.419998
2020-03-03,36.820000
2020-03-04,31.990000
2020-03-05,39.619999
2020-03-06,41.939999
...,...
2024-02-23,13.750000
2024-02-26,13.740000
2024-02-27,13.430000


## Feature Variable 5: Layoffs

We will get this data from layoffs.fyi, a page which tracks layoffs in the technology sector. But the data is in fragments, and does not follow the same format as the other features. <br>

So we need to perform additional operations to interpolate it and bring it into a format which is convenient for us.

In [20]:
layoffs_2024 = pd.read_csv("layoffs_data_2024.csv")

Lets get a broad overview of how the data looks like

In [21]:
layoffs_2024

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off
0,Fisker,Los Angeles,Transportation,0.15,2024-02-29,https://techcrunch.com/2024/02/29/fisker-layof...,1700.0,Post-IPO,2024-02-29 23:35:20,United States,NaN,Unknown
1,Electronic Arts,SF Bay Area,Consumer,0.05,2024-02-28,https://www.cnbc.com/2024/02/28/ea-layoffs-com...,2.0,Post-IPO,2024-02-29 00:45:00,United States,670.0,Unknown
2,Vacasa,Portland,Travel,0.05,2024-02-28,https://www.oregonlive.com/business/2024/02/po...,834.0,Post-IPO,2024-02-29 17:22:37,United States,320.0,Unknown
3,Treasury Prime,SF Bay Area,Finance,0.40,2024-02-28,https://www.bankingdive.com/news/treasury-prim...,71.0,Series C,2024-02-29 00:48:03,United States,40.0,Unknown
4,Motional,Boston,Transportation,0.05,2024-02-28,https://techcrunch.com/2024/02/28/hyundai-back...,NaN,Unknown,2024-02-28 18:50:58,United States,NaN,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
3480,Service,Los Angeles,Travel,1.00,2020-03-16,https://techcrunch.com/2020/03/16/travel-savin...,5.1,Seed,2020-08-31 15:17:26,United States,NaN,Unknown
3481,HopSkipDrive,Los Angeles,Transportation,0.10,2020-03-13,https://layoffs.fyi/2020/04/02/hopskipdrive-la...,45.0,Unknown,2020-04-03 04:47:03,United States,8.0,Unknown
3482,Panda Squad,SF Bay Area,Consumer,0.75,2020-03-13,https://twitter.com/danielsinger/status/123854...,1.0,Seed,2020-04-17 00:12:49,United States,6.0,Unknown
3483,Tamara Mellon,Los Angeles,Retail,0.40,2020-03-12,https://layoffs.fyi/list/tamara-mellon/,90.0,Series C,2020-03-31 16:04:23,United States,20.0,Unknown


Let us first get rid of unnecessary columns.

In [22]:
layoffs_2024 = layoffs_2024[['Company', 'Date', 'Laid_Off_Count', 'Percentage', 'Stage']]

Now we see there are missing values. Let's further analyze the statistics in order to decide what approach to take in order to deal with them.

In [23]:
layoffs_2024.isna().sum()

Company              0
Date                 0
Laid_Off_Count    1195
Percentage        1240
Stage                0
dtype: int64

In [24]:
layoffs_2024.describe()

,Laid_Off_Count,Percentage
count,2290.000000,2245.000000
mean,250.860699,0.278106
std,780.128866,0.283833
min,3.000000,0.000000
25%,40.000000,0.100000
50%,80.000000,0.170000
75%,185.750000,0.300000
max,12000.000000,1.000000


Around 33% of the values are missing for laid off count, the value of our interest.<br>
Let us replace those values by median values for the companies of the same stage. <br>

We making an assumption here, that the number of employees of the companies at different stages are similar, and typically companies layoff 10-20% of their workforce. So a median value would be a reasonable guess to replace null values.

In [25]:
layoffs_2024[['Stage', 'Laid_Off_Count']].groupby(by=['Stage']).describe()

Laid_Off_Count                                                 \
                        count        mean          std    min     25%    50%   
Stage                                                                          
Acquired                200.0  286.670000   537.994621    7.0   54.75  100.0   
Post-IPO                515.0  605.578641  1463.887681   11.0   80.00  200.0   
Private Equity           35.0  299.971429   576.720775   10.0   39.50  100.0   
Seed                     44.0   49.068182    58.298110    3.0   13.00   29.5   
Series A                142.0   56.464789    67.146359    4.0   19.00   30.0   
Series B                266.0  107.082707   497.790417    5.0   28.00   46.0   
Series C                253.0   94.387352   143.880324    5.0   29.00   50.0   
Series D                216.0  110.319444   103.519619    7.0   43.75   83.0   
Series E                123.0  149.764228   255.196166   14.0   60.00   90.0   
Series F                 72.0  179.416667   198.290990    5.0   73.00  120.0   
Series G                 14.0  299.785714   377.581612   60.0   88.00  175.0   
Series H                 27.0  293.740741   349.952150   20.0  100.00  170.0   
Series I                  7.0  407.857143   401.454202   35.0  105.00  350.0   
Series J                  7.0  510.000000   503.686410  100.0  175.00  200.0   
Subsidiary               17.0  218.470588   295.260080   15.0   40.00  100.0   
Unknown                 352.0  155.650568   316.910594    5.0   34.75   69.5   

                                 
                   75%      max  
Stage                            
Acquired        276.25   4375.0  
Post-IPO        472.50  12000.0  
Private Equity  147.00   2500.0  
Seed             63.75    300.0  
Series A         70.00    400.0  
Series B         85.00   8000.0  
Series C        100.00   1500.0  
Series D        136.50    800.0  
Series E        150.00   2500.0  
Series F        231.75   1300.0  
Series G        337.75   1500.0  
Series H        325.00   1500.0  
Series I        580.00   1100.0  
Series J        760.00   1400.0  
Subsidiary      280.00   1000.0  
Unknown         140.00   3000.0

The data distribution is quite skewed, so replacing the null values with median seems to be the preferred approach.

In [26]:
layoffs_2024_grouped_by_stage = layoffs_2024[['Stage', 'Laid_Off_Count']].groupby(by=['Stage']).agg('median')
layoff_2024_count_dict = layoffs_2024_grouped_by_stage.to_dict()['Laid_Off_Count']
layoff_2024_count_dict

{'Acquired': 100.0,
 'Post-IPO': 200.0,
 'Private Equity': 100.0,
 'Seed': 29.5,
 'Series A': 30.0,
 'Series B': 46.0,
 'Series C': 50.0,
 'Series D': 83.0,
 'Series E': 90.0,
 'Series F': 120.0,
 'Series G': 175.0,
 'Series H': 170.0,
 'Series I': 350.0,
 'Series J': 200.0,
 'Subsidiary': 100.0,
 'Unknown': 69.5}

Now, we need to substitute these values in the null values in the dataframe.

In [27]:
stage_median = layoffs_2024.groupby('Stage')['Laid_Off_Count'].transform('median')
layoffs_2024['Laid_Off_Count'] = layoffs_2024['Laid_Off_Count'].fillna(stage_median)
layoffs_2024['Laid_Off_Count'] = layoffs_2024['Laid_Off_Count'].astype(int)

/var/folders/fm/jxt3g9_s4ps7mv9cq1d2f2h80000gn/T/ipykernel_81672/1998710801.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  layoffs_2024['Laid_Off_Count'] = layoffs_2024['Laid_Off_Count'].fillna(stage_median)
/var/folders/fm/jxt3g9_s4ps7mv9cq1d2f2h80000gn/T/ipykernel_81672/1998710801.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  layoffs_2024['Laid_Off_Count'] = layoffs_2024['Laid_Off_Count'].astype(int)


Let us remove other unnecessary columns now.

In [28]:
layoffs_2024 = layoffs_2024[['Date', 'Laid_Off_Count']]
layoffs_2024

,Date,Laid_Off_Count
0,2024-02-29,200
1,2024-02-28,670
2,2024-02-28,320
3,2024-02-28,40
4,2024-02-28,69
...,...,...
3480,2020-03-16,29
3481,2020-03-13,8
3482,2020-03-13,6
3483,2020-03-12,20


We have multiple layoff entries on a single day, we need to aggregate them. And we need to set date as index.

In [29]:
layoffs_2024 = layoffs_2024.groupby('Date')['Laid_Off_Count'].sum().reset_index()
# Change index to date
layoffs_2024.set_index('Date', inplace=True)

In [30]:
layoffs_2024

,Laid_Off_Count
Date,
2020-03-11,75
2020-03-12,20
2020-03-13,14
2020-03-16,175
2020-03-18,4
...,...
2024-02-23,133
2024-02-26,1639
2024-02-27,1529


## Putting it all together

We have different values of rows for different variables. We need to bring them to parity. <br>
T100 Index, US currency Index, Volatility Index : 1007 rows(2 Mar 2020 - 29 Feb 2024) <br>
Short term interest rates, Long Term interest rates: 1045 rows(2 Mar 2020 - 29 Feb 2024) <br>
Layoffs: 762 rows(11 Mar 2020 - 29 Feb 2024)

### Dealing with layoff data

Lets first deal with layoff data.  <br>
1. We will check if there is any layoff data on weekends, are they are non-trading days. <br>
2. If there are some dates for which layoff data is missing, those days are when layoffs didn't happen, so its value should be kept as 0 in those days.

In [31]:
layoffs_2024.index

Index(['2020-03-11', '2020-03-12', '2020-03-13', '2020-03-16', '2020-03-18',
       '2020-03-19', '2020-03-20', '2020-03-23', '2020-03-24', '2020-03-25',
       ...
       '2024-02-15', '2024-02-16', '2024-02-20', '2024-02-21', '2024-02-22',
       '2024-02-23', '2024-02-26', '2024-02-27', '2024-02-28', '2024-02-29'],
      dtype='object', name='Date', length=762)

In [32]:
start_date = '2020-03-01'
end_date = '2020-03-10'

filler_date_range = pd.date_range(start=start_date, end=end_date, freq = 'D')
layoffs_2024_filler = pd.DataFrame(index=filler_date_range)
layoffs_2024_filler["Laid_Off_Count"] = 0

In [33]:
layoffs_2024_filler.index = layoffs_2024_filler.index.date
layoffs_2024 = pd.concat([layoffs_2024_filler, layoffs_2024], axis=0)
layoffs_2024.index = pd.to_datetime(layoffs_2024.index)

In [34]:
layoffs_2024.index

DatetimeIndex(['2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
               '2020-03-09', '2020-03-10',
               ...
               '2024-02-15', '2024-02-16', '2024-02-20', '2024-02-21',
               '2024-02-22', '2024-02-23', '2024-02-26', '2024-02-27',
               '2024-02-28', '2024-02-29'],
              dtype='datetime64[ns]', length=772, freq=None)

In [35]:
layoffs_2024 = layoffs_2024.resample('D').asfreq().fillna(0)

In [36]:
layoffs_2024

,Laid_Off_Count
2020-03-01,0.0
2020-03-02,0.0
2020-03-03,0.0
2020-03-04,0.0
2020-03-05,0.0
...,...
2024-02-25,0.0
2024-02-26,1639.0
2024-02-27,1529.0
2024-02-28,1099.0


In [37]:
layoffs_2024.index = pd.to_datetime(layoffs_2024.index)
weekend_dates = layoffs_2024.index[layoffs_2024.index.weekday >=5]

if len(weekend_dates) > 0:
    print("Dates in the index that fall on a weekend are:")
    print(weekend_dates)
else:
    print("There are no dates in the index that fall on a weekend.")

Dates in the index that fall on a weekend are:
DatetimeIndex(['2020-03-01', '2020-03-07', '2020-03-08', '2020-03-14',
               '2020-03-15', '2020-03-21', '2020-03-22', '2020-03-28',
               '2020-03-29', '2020-04-04',
               ...
               '2024-01-27', '2024-01-28', '2024-02-03', '2024-02-04',
               '2024-02-10', '2024-02-11', '2024-02-17', '2024-02-18',
               '2024-02-24', '2024-02-25'],
              dtype='datetime64[ns]', length=417, freq=None)


So we have a lot of weekends where layoffs were reported. First lets fill all the missing days with 0 values. <br>
Post that, we will push the weekend layoff values to the next Monday, as the impact of layoff news on weekends is seen on the coming Monday.

In [38]:
if len(weekend_dates) > 0:
    # Iterate over each weekend date
    for weekend_date in weekend_dates:
        # Get the value corresponding to the weekend date
        weekend_value = layoffs_2024.loc[weekend_date, 'Laid_Off_Count']
        
        # Calculate the next Monday
        next_monday = weekend_date + pd.DateOffset(weekday=0)
        
        # Add the weekend value to the value of the next Monday
        if next_monday in layoffs_2024.index:
            layoffs_2024.loc[next_monday, 'Laid_Off_Count'] += weekend_value
        else:
            # If next Monday doesn't exist in the index, create a new row
            layoffs_2024.loc[next_monday] = weekend_value
    
    # Drop the rows corresponding to the weekend dates
    layoffs_2024.drop(weekend_dates, inplace=True)
    
    print("Values corresponding to weekend dates have been aggregated and added to the value of the next Monday.")
    print("Updated DataFrame:")
    print(layoffs_2024)
else:
    print("There are no dates in the index that fall on a weekend.")

Values corresponding to weekend dates have been aggregated and added to the value of the next Monday.
Updated DataFrame:
            Laid_Off_Count
2020-03-02             0.0
2020-03-03             0.0
2020-03-04             0.0
2020-03-05             0.0
2020-03-06             0.0
...                    ...
2024-02-23           133.0
2024-02-26          1639.0
2024-02-27          1529.0
2024-02-28          1099.0
2024-02-29           200.0

[1044 rows x 1 columns]


We still have some extra entries, this is because of trading holidays. Let us deal with that. We will push all values of layoffs on trading days to the next trading day.

In [39]:
trading_holidays = [
    "2020-04-10",
    "2020-05-25",
    "2020-07-03",
    "2020-09-07",
    "2020-11-26",
    "2020-12-25",
    "2021-01-01",
    "2021-01-18",
    "2021-02-15",
    "2021-04-02",
    "2021-05-31",
    "2021-07-05",
    "2021-09-06",
    "2021-11-25",
    "2021-12-24",
    "2022-01-17",
    "2022-02-21",
    "2022-04-15",
    "2022-05-30",
    "2022-06-20",
    "2022-07-04",
    "2022-09-05",
    "2022-11-24",
    "2022-12-26",
    "2023-01-02",
    "2023-01-16",
    "2023-02-20",
    "2023-04-07",
    "2023-05-29",
    "2023-06-19",
    "2023-07-04",
    "2023-09-04",
    "2023-11-23",
    "2023-12-25",
    "2024-01-01",
    "2024-01-15",
    "2024-02-19"
]

In [40]:
from pandas.tseries.offsets import CustomBusinessDay

# Create a DataFrame with sample data and set the index as datetime
# Assuming your DataFrame is named df
# Replace this with your actual DataFrame and data
data = {'value': range(len(layoffs_2024))}
layoffs_2024_1 = pd.DataFrame(data, index=pd.to_datetime(layoffs_2024.index))

# Define a CustomBusinessDay with US trading holidays
us_trading_calendar = CustomBusinessDay(holidays=trading_holidays)

# Identify the next valid trading day for each holiday
next_trading_days = layoffs_2024.index.map(lambda x: x + us_trading_calendar)

# Move the values to the next valid trading day
for holiday, next_trading_day in zip(trading_holidays, next_trading_days):
    if holiday in layoffs_2024.index and next_trading_day in layoffs_2024.index:
        layoffs_2024.loc[next_trading_day] += layoffs_2024.loc[holiday]
        layoffs_2024.drop(index=holiday, inplace=True)

# Sort the DataFrame by index
layoffs_2024.sort_index(inplace=True)

# Display the DataFrame after moving values to the next valid trading day
print(layoffs_2024)

            Laid_Off_Count
2020-03-02             0.0
2020-03-03           391.0
2020-03-04           200.0
2020-03-05             0.0
2020-03-06            46.0
...                    ...
2024-02-23           133.0
2024-02-26          1639.0
2024-02-27          1529.0
2024-02-28          1099.0
2024-02-29           200.0

[1007 rows x 1 columns]


### Dealing with Interest rates data

The interest rate dataframes have 1045 rows, instead of 1007. We will simply drop those extra entries.

In [41]:
short_term_rates_df = short_term_rates_df.loc[layoffs_2024.index]
long_term_rates_df = long_term_rates_df.loc[layoffs_2024.index]

In [42]:
short_term_rates_df

,DTB3
2020-03-02,1.10
2020-03-03,0.93
2020-03-04,0.71
2020-03-05,0.61
2020-03-06,0.45
...,...
2024-02-23,5.26
2024-02-26,5.26
2024-02-27,5.25
2024-02-28,5.24


In [43]:
long_term_rates_df

,DGS10
2020-03-02,1.10
2020-03-03,1.02
2020-03-04,1.02
2020-03-05,0.92
2020-03-06,0.74
...,...
2024-02-23,4.26
2024-02-26,4.28
2024-02-27,4.31
2024-02-28,4.27


### Organizing and Concatenating the datasets

In [44]:
nasdaq_t100_prices = nasdaq_t100_prices.rename(columns={"Adj Close": "nasdaq_t100"})
nasdaq_t100_prices

,nasdaq_t100
Date,
2020-03-02,5314.009766
2020-03-03,5138.990234
2020-03-04,5367.529785
2020-03-05,5197.419922
2020-03-06,5086.419922
...,...
2024-02-23,10265.709961
2024-02-26,10329.919922
2024-02-27,10347.509766


In [45]:
short_term_rates_df = short_term_rates_df.rename(columns={"DTB3": "short_term_rates"})
short_term_rates_df

,short_term_rates
2020-03-02,1.10
2020-03-03,0.93
2020-03-04,0.71
2020-03-05,0.61
2020-03-06,0.45
...,...
2024-02-23,5.26
2024-02-26,5.26
2024-02-27,5.25
2024-02-28,5.24


In [46]:
long_term_rates_df = long_term_rates_df.rename(columns={"DGS10": "long_term_rates"})
long_term_rates_df

,long_term_rates
2020-03-02,1.10
2020-03-03,1.02
2020-03-04,1.02
2020-03-05,0.92
2020-03-06,0.74
...,...
2024-02-23,4.26
2024-02-26,4.28
2024-02-27,4.31
2024-02-28,4.27


In [47]:
dollar_index_values = dollar_index_values.rename(columns={"Adj Close": "dollar_index"})
dollar_index_values

,dollar_index
Date,
2020-03-02,97.360001
2020-03-03,97.150002
2020-03-04,97.339996
2020-03-05,96.820000
2020-03-06,95.949997
...,...
2024-02-23,103.940002
2024-02-26,103.830002
2024-02-27,103.830002


In [48]:
vix_index_values = vix_index_values.rename(columns={"Adj Close": "vix_index"})
vix_index_values

,vix_index
Date,
2020-03-02,33.419998
2020-03-03,36.820000
2020-03-04,31.990000
2020-03-05,39.619999
2020-03-06,41.939999
...,...
2024-02-23,13.750000
2024-02-26,13.740000
2024-02-27,13.430000


In [49]:
dataset = pd.concat([nasdaq_t100_prices, short_term_rates_df, long_term_rates_df, dollar_index_values, vix_index_values, layoffs_2024], axis=1)
dataset

,nasdaq_t100,short_term_rates,long_term_rates,dollar_index,vix_index,Laid_Off_Count
2020-03-02,5314.009766,1.10,1.10,97.360001,33.419998,0.0
2020-03-03,5138.990234,0.93,1.02,97.150002,36.820000,391.0
2020-03-04,5367.529785,0.71,1.02,97.339996,31.990000,200.0
2020-03-05,5197.419922,0.61,0.92,96.820000,39.619999,0.0
2020-03-06,5086.419922,0.45,0.74,95.949997,41.939999,46.0
...,...,...,...,...,...,...
2024-02-23,10265.709961,5.26,4.26,103.940002,13.750000,133.0
2024-02-26,10329.919922,5.26,4.28,103.830002,13.740000,1639.0
2024-02-27,10347.509766,5.25,4.31,103.830002,13.430000,1529.0
2024-02-28,10257.580078,5.24,4.27,103.980003,13.840000,1099.0


In [50]:
dataset.to_csv('dataset.csv')